# Expected failure: asyncio + ProcessPoolExecutor (spawn) with notebook-defined worker


In [ ]:
# pytest-nb-as-test: must-raise-exception=True
import asyncio
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor

expected_error_seen = False


def cpu_fn(x):
    return x * x


async def main() -> None:
    ctx = mp.get_context("spawn")
    loop = asyncio.get_running_loop()
    xs = [1, 2, 3]
    with ProcessPoolExecutor(max_workers=2, mp_context=ctx) as ex:
        tasks = [loop.run_in_executor(ex, cpu_fn, x) for x in xs]
        return await asyncio.gather(*tasks)


try:
    out = await main()
    raise RuntimeError(f"unexpected success, out={out!r}")
except Exception as e:
    msg = repr(e) + "\n" + str(e)
    expected_error_seen = any(
        s in msg
        for s in [
            "Can't pickle",
            "cannot pickle",
            "Can't get attribute",
            "BrokenProcessPool",
            "ModuleNotFoundError",
        ]
    )
    raise

In [ ]:
assert expected_error_seen is True